In [1]:
import re
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 78% !important; }</style>"))

In [3]:
def getRegex(df : pd.DataFrame, col_name : str, regex : str) -> pd.DataFrame :
    return df[df[col_name].str.contains(regex)]

# 반복 찾기

## 하나 이상의 문자 찾기(+)

* 문자 집합에 +를 사용한다면?
    * [0-9+]가 아니라 [0-9]+를 사용해야 한다. 왜냐하면, 문자 집합은 or조건이기 때문이다.

In [4]:
text1 = """Send personal email to ben@forta.com. For Questions
about a book user support@forta.com. Feel free to send
unsolicited email to spam@forta.com (wouldn`t it be
nice if it were the simple, huh?)."""

In [5]:
re.findall(r"\w+@\w+\.com", text1)

['ben@forta.com', 'support@forta.com', 'spam@forta.com']

In [11]:
text2 = """Send personal email to ben@forta.com. or ben.forta@forta.com
For Questions about a book user support@forta.com. 
If your message is urgent try ben@urgent.forta.com.
Feel free to send unsolicited email to spam@forta.com (wouldn`t it be
nice if it were the simple, huh?)."""

In [12]:
re.findall(r"\w+@\w+\.\w+", text2)

['ben@forta.com',
 'forta@forta.com',
 'support@forta.com',
 'ben@urgent.forta',
 'spam@forta.com']

* 제대로 검색되지 않은 주소들이 있음. 왜?
    * \w는 .(마침표)를 포함하지 않는다.
    * 따라서 마침표 집합을 정의해야 한다.

In [24]:
#참고 - 문자클래스안에 메타 문자 .이 들어가게 되면 마침표로 인식된다. 이스케이프를 해도 똑같다.
re.findall(r"[.]", 'asd....')

['.', '.', '.', '.']

In [30]:
re.findall(r"[\w.]+@[\w.]+\.\w+", text2)

['ben@forta.com',
 'ben.forta@forta.com',
 'support@forta.com',
 'ben@urgent.forta.com',
 'spam@forta.com']

## 문자가 없거나 하나 이상 연속하는 문자 찾기(*)

* (+)와 (*)의 차이 
    * (+)는 문자나 집합이 최소한 하나 이상 일치해야 한다.
    * (*)은 문자나 집합이 0개 이상 일치하면 된다.
    * 결론적으로 반드시 해당 문자가 있어야 한다면 + 그렇지 않다면 *이다.

In [31]:
text3 = "Hello .ben@forta.com is my email address."

In [32]:
re.search(r"[\w.]+@[\w.]+\.\w+", text3)

<re.Match object; span=(6, 20), match='.ben@forta.com'>

In [36]:
#.마침표를 빼려면?
re.search(r"\w+[\w.]*@[\w.]+\.\w+", text3)

<re.Match object; span=(7, 20), match='ben@forta.com'>

## 문자가 없거나 하나인 문자 찾기(?)

* ?는 문자나 집합이 없거나 한번만 있을 때 유용하다.

[메타문자리스트](https://docs.trifacta.com/display/SS/Supported+Special+Regular+Expression+Characters)

In [37]:
text4 = """The URL is http://www.forta.com/, to connect
securely use https://www.forta.com/ instead."""

In [39]:
re.findall(r"http:\/\/[\w.]+\/", text4)

['http://www.forta.com/']

In [49]:
#http와 https두개를 일치 시키려면?
print("방법1", re.findall(r"https?:\/\/[\w.]+\/", text4))
print("방법2", re.findall(r"http[\w]*:\/\/[\w.]+\/", text4))
print("방법3", re.findall(r"http[s]?:\/\/[\w.]+\/", text4))
print("방법3", re.findall(r"http[s]*:\/\/[\w.]+\/", text4)) #지금 상황은 ?를 쓰는게 더 적절

방법1 ['http://www.forta.com/', 'https://www.forta.com/']
방법2 ['http://www.forta.com/', 'https://www.forta.com/']
방법3 ['http://www.forta.com/', 'https://www.forta.com/']
방법3 ['http://www.forta.com/', 'https://www.forta.com/']


## 구간 지정하기({})

* +, *, ? 는 일치하는 문자 수에 제한이 없다!
* 즉, 일치하는 문자수를 명시적으로 제한할 수 없다!

In [4]:
text5 = """body {
    background-color; #fefbd8;
}
h1 {
    background-color; #0000ff;
}
div {
    background-color; #d0f4e6;
}
span {
    background-color; #f08970;
}
"""

In [6]:
re.findall("#[A-Fa-f0-9]{6}", text5)

['#fefbd8', '#0000ff', '#d0f4e6', '#f08970']

## 범위 구간 찾기 ({d1,d2})

In [10]:
text6 = """4/8/17
10-6-2018
2/2/2
01-01-01
"""

In [12]:
#참고로 이 범위 구간을 찾을 때 띄어쓰기를 하면 안된다.
re.findall("\d{1,2}[-\/]\d{1,2}[-\/]\d{2,4}", text6)

['4/8/17', '10-6-2018', '01-01-01']

## 최소 구간 찾기({d1,})

In [13]:
text7 = """1001: $496.80
1002: $1290.69
1003: $26.43
1004: $613.42
1005: $7.61
1006: $414.90
1007: $25.00
"""

In [15]:
#달러도 메타 문자다!
re.findall(r"\d+: \$\d{3,}\.\d{2}", text7)

['1001: $496.80', '1002: $1290.69', '1004: $613.42', '1006: $414.90']

## 과하게 일치하는 상황 방지하기 (greedy search)

* *와 +같은 메타 문자는 탐욕적이므로 가능한 가장 큰 덩어리를 찾으려 한다.
* 과자1과 과자2 합쳐서 과자 __ 과자 먹어랬더니 다 먹는 경우

In [16]:
text8 = """This offer is not available to customers
living in <b>AK</b> and <b>HI</b>.
"""

In [23]:
re.findall(r"<[Bb]>.*<\/[Bb]>", text8)

['<b>AK</b> and <b>HI</b>']

* 여기서 탐욕적이라는것은 < b >AK < /b > 를 찾은 것이 아니고 더 큰 덩어리를 찾았다는 뜻이다.
* 이는 *, + 자체가 탐욕적인 탐색을 위해 설계되었기 때문이다.

* 탐욕적 수량자 : *, +, {n,}
* 게으른 수량자 : *?, +?, {n,}?

In [25]:
re.findall(r"<[Bb]>.*?<\/[Bb]>", text8)

['<b>AK</b>', '<b>HI</b>']